In [1]:
from nuplan.common.maps.nuplan_map.map_factory import get_maps_api, get_maps_db

MAP_LOCATIONS = {"sg-one-north", "us-ma-boston", "us-nv-las-vegas-strip", "us-pa-pittsburgh-hazelwood"}

In [2]:
import geopandas as gpd
import pyogrio


# Path to your .gpkg file
path_to_gpkg = "/home/daniel/nuplan/dataset/maps/us-nv-las-vegas-strip/9.15.1915/map.gpkg"
# path_to_gpkg = "/home/daniel/nuplan/dataset/maps/us-ma-boston/9.12.1817/map.gpkg"
# path_to_gpkg = "/home/daniel/nuplan/dataset/maps/us-pa-pittsburgh-hazelwood/9.17.1937/map.gpkg"
# path_to_gpkg = "/home/daniel/nuplan/dataset/maps/sg-one-north/9.17.1964/map.gpkg"


# List all available layers
layers = [
    "baseline_paths",
    "carpark_areas",
    "generic_drivable_areas",
    "dubins_nodes",
    "lane_connectors",
    "intersections",
    "boundaries",
    "crosswalks",
    "lanes_polygons",
    "lane_group_connectors",
    "lane_groups_polygons",
    "walkways",
    "gen_lane_connectors_scaled_width_polygons",
    "meta",
]


def load_layer(layer_name: str) -> gpd.geodataframe:


    map_meta = gpd.read_file(path_to_gpkg, layer="meta", engine="pyogrio")
    projection_system = map_meta[map_meta["key"] == "projectedCoordSystem"]["value"].iloc[0]

    gdf_in_pixel_coords = pyogrio.read_dataframe(path_to_gpkg, layer=layer_name, fid_as_index=True)
    gdf_in_utm_coords = gdf_in_pixel_coords.to_crs(projection_system)

    # For backwards compatibility, cast the index to string datatype.
    #   and mirror it to the "fid" column.
    gdf_in_utm_coords.index = gdf_in_utm_coords.index.map(str)
    gdf_in_utm_coords["fid"] = gdf_in_utm_coords.index

    return gdf_in_utm_coords

pyogrio.read_dataframe(path_to_gpkg)

/home/daniel/anaconda3/envs/asim/lib/python3.9/site-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'map.gpkg': 'baseline_paths' (default), 'carpark_areas', 'generic_drivable_areas', 'dubins_nodes', 'lane_connectors', 'intersections', 'boundaries', 'crosswalks', 'lanes_polygons', 'lane_group_connectors', 'lane_groups_polygons', 'road_segments', 'stop_polygons', 'traffic_lights', 'walkways', 'gen_lane_connectors_scaled_width_polygons', 'meta'. Specify layer parameter to avoid this warning.
  result = read_func(


,path_type_fid,creator_id,lane_fid,lane_connector_fid,dubins_node_fids,geometry
0,1,004604d5-06ec-45e9-b0a0-2999ae9663ef,NaN,69256.0,"41728,46497","LINESTRING (-115.17207 36.10806, -115.17207 36..."
1,0,005c1777-ba68-4844-b704-d89c5c608712,NaN,70517.0,"44420,45509","LINESTRING (-115.15451 36.06824, -115.15451 36..."
2,0,0062f99a-14e3-4cd9-98b2-832c8de003db,NaN,70564.0,"45034,44743","LINESTRING (-115.17281 36.10494, -115.17281 36..."
3,0,006ef330-e027-407a-9e5b-b75b4f2ba945,NaN,68815.0,"46166,44987,43737","LINESTRING (-115.17196 36.12552, -115.17196 36..."
4,0,007a12d9-1dac-498e-96b1-71fd7663c471,NaN,70001.0,"43312,46139","LINESTRING (-115.17305 36.09213, -115.17305 36..."
...,...,...,...,...,...,...
4578,0,ff88c5cc-b01f-4abe-bfce-59f3848231d2,68381.0,NaN,"45594,45470","LINESTRING (-115.15279 36.06789, -115.1528 36...."
4579,0,ffa0df14-2c33-4464-b0ff-24f88cb3ee8f,NaN,68877.0,"43009,44986","LINESTRING (-115.17248 36.07156, -115.17248 36..."
4580,1,ffbe9b65-b141-440b-835d-c1001cf00c72,67619.0,NaN,"44252,41512","LINESTRING (-115.17256 36.09726, -115.17256 36..."
4581,0,ffd2abe6-3c44-45bc-87bf-02a1ff793e13,NaN,69729.0,"42352,43475","LINESTRING (-115.17296 36.09386, -115.17296 36..."


In [7]:
[
    "carpark_areas",
    "generic_drivable_areas",
    "dubins_nodes",
    "lane_connectors",
    "intersections",
    "boundaries",
    "crosswalks",
    "lanes_polygons",
    "lane_group_connectors",
    "lane_groups_polygons",
    "road_segments",
    "stop_polygons",
    "traffic_lights",
    "walkways",
    "gen_lane_connectors_scaled_width_polygons",
    "meta",
]

lane_connectors = load_layer("lane_connectors")

lane_connectors[lane_connectors["fid"] == "64127"]

,exit_lane_group_fid,exit_lane_fid,entry_lane_group_fid,lane_group_connector_fid,entry_lane_fid,intersection_fid,turn_type_fid,bulb_fids,traffic_light_stop_line_fids,overlap,creator_id,reflector_side_fid,left_has_reflectors,right_has_reflectors,speed_limit_mps,geometry,fid
fid,,,,,,,,,,,,,,,,,
64127,65536,63193,46931,60107,67321,60104,1,"46930,46932,46933,48889,48890,48891,46935,4693...",63191,NaN,31478f7a-4521-46cf-baf4-fce0ce509aaf,1,True,False,6.705409,"LINESTRING (664450.633 3997796.554, 664450.627...",64127


In [ ]:

import pandas as pd
lane_df =  load_layer("generic_drivable_areas")
lane_df

In [ ]:
walkways_df = load_layer("carparks")
walkways_df

In [ ]:

gen_lane_connectors_scaled_width_polygons_df = load_layer("boundaries")

gen_lane_connectors_scaled_width_polygons_df[gen_lane_connectors_scaled_width_polygons_df["fid"] == "17950"]

In [ ]:
[None] * 10

In [ ]:
import pandas as pd
lane_df =  load_layer("baseline_paths")
lane_df

In [ ]:
def mps_to_mph(mph: float) -> float:
    """
    Convert meters per second to miles per hour.
    :param mph: miles per hour [mi/h]
    :return: meters per second [m/s]
    """
    return mph / 0.44704
mps_to_mph(6.705409029950827)

In [ ]:
# fix, ax = plt.subplots(figsize=(10, 10))

# type = 3

# for i in np.random.choice(len(geoms[type]), 10):
#     ax.plot(*geoms[type][i].coords.xy, color="blue")


In [ ]:
geoms[2][1]

In [ ]:
geoms[3][1]